# Advanced Dataframes

## Part 3 - Reshaping
- crosstab
- pivot_table

In [1]:
import pandas as pd
import numpy as np

np.random.seed(123)

In [2]:
# Create list of values for names column.
students = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas',
            'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']

# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.
math_grades = np.random.randint(low=60, high=100, size=len(students))
english_grades = np.random.randint(low=60, high=100, size=len(students))
reading_grades = np.random.randint(low=60, high=100, size=len(students))

# Randomly generate if a student is in classroom A or classroom B
classroom = np.random.choice(['A1', 'B1'], len(students))

df = pd.DataFrame({'name':students,
                   'math':math_grades,
                   'reading':reading_grades,
                   'english':english_grades,
                   'room':classroom})

#calculated overall grade and letter
df['overall_grade'] = df[['math','reading','english']].mean(axis=1).round(1)

#np.where(condition, output if true, output if false)
df['overall_letter'] = np.where(df.overall_grade >= 90, 'A', 
                                np.where(df.overall_grade >= 80, 'B','C'))

df

,name,math,reading,english,room,overall_grade,overall_letter
0,Sally,62,80,85,A1,75.7,C
1,Jane,88,67,79,B1,78.0,C
2,Suzie,94,95,74,A1,87.7,B
3,Billy,98,88,96,B1,94.0,A
4,Ada,77,98,92,A1,89.0,B
5,John,79,93,76,B1,82.7,B
6,Thomas,82,81,64,A1,75.7,C
7,Marie,93,90,63,A1,82.0,B
8,Albert,92,87,62,A1,80.3,B
9,Richard,69,94,80,A1,81.0,B


## Using `.crosstab`
COUNT the number of occurances between each subgroup
- format: `pd.crosstab(index=_, column=_)`

In [3]:
df.head()

,name,math,reading,english,room,overall_grade,overall_letter
0,Sally,62,80,85,A1,75.7,C
1,Jane,88,67,79,B1,78.0,C
2,Suzie,94,95,74,A1,87.7,B
3,Billy,98,88,96,B1,94.0,A
4,Ada,77,98,92,A1,89.0,B


#### find all counts of the room and overall letter grade

In [7]:
pd.crosstab(df.room, df.overall_letter)

overall_letter,A,B,C
room,,,
A1,0,5,3
B1,2,1,1


In [8]:
pd.crosstab(df.overall_letter, df.room)

room,A1,B1
overall_letter,,
A,0,2
B,5,1
C,3,1


#### add total counts

In [9]:
#margins argument
pd.crosstab(df.room, df.overall_letter, margins=True)

overall_letter,A,B,C,All
room,,,,
A1,0,5,3,8
B1,2,1,1,4
All,2,6,4,12


#### give percentages instead of counts

In [13]:
#normalize argument
pd.crosstab(df.room, df.overall_letter, normalize=True).round(2)

overall_letter,A,B,C
room,,,
A1,0.00,0.42,0.25
B1,0.17,0.08,0.08


In [14]:
pd.crosstab(df.room, df.overall_letter, normalize='index').round(2)

overall_letter,A,B,C
room,,,
A1,0.0,0.62,0.38
B1,0.5,0.25,0.25


In [15]:
pd.crosstab(df.room, df.overall_letter, normalize='columns').round(2)

overall_letter,A,B,C
room,,,
A1,0.0,0.83,0.75
B1,1.0,0.17,0.25


### Using `.pivot_table`

creates a summary table similar to excel pivot table 

need to include 3 of the following:
- which values will make up the rows (the index)
- which values will make up the columns
- the values we are aggregating
- an aggregation method (aggfunc); if we can omit this, and the average will be used by default

format: `pd.pivot_table(data=_, index=_, columns=_, values=_, aggfunc=_)`

#### show the average math grade by overall letter grade and room

In [20]:
pd.pivot_table(data=df, index='room', columns='overall_letter',
              values='math')

overall_letter,A,B,C
room,,,
A1,NaN,85.0,78.666667
B1,95.0,79.0,88.000000


In [19]:
df [df.overall_letter == 'B']

,name,math,reading,english,room,overall_grade,overall_letter
2,Suzie,94,95,74,A1,87.7,B
4,Ada,77,98,92,A1,89.0,B
5,John,79,93,76,B1,82.7,B
7,Marie,93,90,63,A1,82.0,B
8,Albert,92,87,62,A1,80.3,B
9,Richard,69,94,80,A1,81.0,B


#### show the MAX math grade by overall letter grade and room

In [23]:
pd.pivot_table(data=df, index='room', columns='overall_letter',
              values='math', aggfunc='max')

overall_letter,A,B,C
room,,,
A1,NaN,94.0,92.0
B1,98.0,79.0,88.0


> NOTE: `index` and `columns` should be categorical values, and `values` should be numeric

### Using `.map`

Use a dictionary to assign new values based on current values
1. create a dictionary of new values based on old
2. use map to apply it to df

- format: `.map({'old_value':'new_value'})`

#### create new df of restuarant data

In [24]:
n = 40

df = pd.DataFrame({
    'drink': np.random.choice(['Tea', 'Water', 'Water'], n),
    
    'meal': np.random.choice(['Curry', 'Yakisoba Noodle', 'Pad Thai'], n),
})

df.sample(10)

,drink,meal
0,Tea,Pad Thai
38,Water,Curry
11,Tea,Curry
4,Tea,Curry
16,Water,Curry
2,Tea,Yakisoba Noodle
22,Water,Yakisoba Noodle
5,Water,Pad Thai
19,Tea,Pad Thai
15,Water,Yakisoba Noodle


#### create dictionary of prices for all current items

In [25]:
#dictionary format: {key:value}
prices = {
    'Yakisoba Noodle': 9,
    'Curry': 11,
    'Pad Thai': 10,
    'Tea': 2,
    'Water': 0,
}
prices

{'Yakisoba Noodle': 9, 'Curry': 11, 'Pad Thai': 10, 'Tea': 2, 'Water': 0}

#### map drink prices and meal prices to create bill
- bill will include price of both

In [32]:
df.drink.head()

0      Tea
1    Water
2      Tea
3    Water
4      Tea
Name: drink, dtype: object

In [37]:
df['bill'] = df.drink.map(prices) + df.meal.map(prices)

In [38]:
df.head()

,drink,meal,bill
0,Tea,Pad Thai,12
1,Water,Pad Thai,10
2,Tea,Yakisoba Noodle,11
3,Water,Pad Thai,10
4,Tea,Curry,13


#### how many orders of each combination of meal and drink are there?

In [39]:
pd.crosstab(df.meal, df.drink)

drink,Tea,Water
meal,,
Curry,3,9
Pad Thai,4,10
Yakisoba Noodle,5,9


#### what percentage of each?

In [40]:
pd.crosstab(df.meal, df.drink, normalize=True)

drink,Tea,Water
meal,,
Curry,0.075,0.225
Pad Thai,0.100,0.250
Yakisoba Noodle,0.125,0.225


#### whats the average bill for each combination?

In [49]:
pd.pivot_table(data=df, index='meal',columns='drink', values='bill', aggfunc='mean')

drink,Tea,Water
meal,,
Curry,13,11
Pad Thai,12,10
Yakisoba Noodle,11,9


In [50]:
pd.pivot_table(data=df, index='meal', values='bill', aggfunc='mean') 
#can group by only a row or only a column

,bill
meal,
Curry,11.500000
Pad Thai,10.571429
Yakisoba Noodle,9.714286


#### we can also get this by using a groupby

In [45]:
df.groupby(['meal','drink']).mean()

bill
meal            drink      
Curry           Tea    13.0
                Water  11.0
Pad Thai        Tea    12.0
                Water  10.0
Yakisoba Noodle Tea    11.0
                Water   9.0